In [1]:
import os
import numpy as np
import pandas as pd
import time
import os.path as osp

# Stage-1: 获取数据

In [12]:
import DataHelper
import importlib
from collections import Counter
importlib.reload(DataHelper)
#np.set_printoptions(threshold=np.inf)

<module 'DataHelper' from 'C:\\Users\\dxeec\\Desktop\\EcoForest-master\\DataHelper.py'>

In [13]:
X_train,y_train, X_sub, y_sub = DataHelper.get_skin_data()

X_train: (25010, 10) y_train: (25010,)
X_test: (100000, 10) y_test: (100000,)


In [14]:
Counter(y_train)

Counter({9: 5,
         8: 5,
         1: 10599,
         0: 12493,
         4: 93,
         3: 513,
         2: 1206,
         5: 54,
         6: 36,
         7: 6})

In [15]:
Counter(y_sub)

Counter({0: 50100,
         1: 42185,
         2: 4873,
         3: 2098,
         5: 195,
         4: 379,
         6: 150,
         7: 18,
         8: 1,
         9: 1})

# Stage-2: 模型应用

In [16]:
import ModelUtils
import importlib
from collections import Counter
importlib.reload(ModelUtils)

<module 'ModelUtils' from 'C:\\Users\\dxeec\\Desktop\\EcoForest-master\\ModelUtils.py'>

In [17]:
from sklearn import metrics
def acc_metrix_mult(a, p):
    return "acc", metrics.accuracy_score(a, p)

# DT, RF, GDBT, XGB 算法

In [18]:
ModelUtils.decision_tree(X_train, y_train, X_sub, y_sub, feval=acc_metrix_mult)

train ('acc', 1.0)
test ('acc', 0.47814)


(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'), array([0, 0, 1, ..., 1, 0, 1]))

In [19]:
ModelUtils.random_forest(X_train, y_train, X_sub, y_sub, n_estimators=200, feval=acc_metrix_mult)

start train: 2019-08-22 15:45:28.770608
start test: 2019-08-22 15:45:29.877865
end test: 2019-08-22 15:45:31.150448
test ('acc', 0.61748)
train ('acc', 1.0)


(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), array([0, 0, 0, ..., 0, 0, 1]))

【实验1】poker在RF的结果。

In [20]:
ModelUtils.gdbt_model(X_train, y_train, X_sub, y_sub, n_estimators=200, feval=acc_metrix_mult)

train ('acc', 0.6518192722910836)
test ('acc', 0.62546)


(GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=200,
               n_iter_no_change=None, presort='auto', random_state=None,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False), array([0, 0, 0, ..., 0, 0, 1]))

# Stage-3：EcoForest

In [21]:
import EnhancedForest_multiclass
import EnhancedForest_producer
import DecomposerForest
import LogUtils
import AlgorithmUtils
import importlib
importlib.reload(DecomposerForest)
importlib.reload(LogUtils)
importlib.reload(EnhancedForest_multiclass)
importlib.reload(EnhancedForest_producer)
importlib.reload(AlgorithmUtils)
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings('ignore')

In [22]:
decoForest = DecomposerForest.DecomposerForest(X_train, y_train, X_sub, y_sub, num_class=10, flag="yeast", isLRStacker=True)
decoForest.set_parameter(criterion="entropy", dropout=0.8, min_samples_leaf=1)
decoForest.fit(n_estimators=200, kfold=2, feval=acc_metrix_mult)


2019-08-22 15:47:13 layer: 1
all data ('acc', 0.9066373450619752)
train loss ('acc', 0.9066373450619752)
pass train loss (0, 0)
pass train loss now (0, 1)
vaild loss ('acc', 0.5909236305477809)
pass vaild loss (0, 0)
pass vaild loss now (0, 1)
test loss ('acc', 0.62338)
pass test loss (0, 0)
pass test loss now (0, 1)
1 [p:0/np:0] 

2019-08-22 15:47:23 layer: 2
all data ('acc', 0.9040383846461415)
[p:324/1:161|n:24686/1:10438] 
train loss ('acc', 0.9040383846461415)
pass train loss ('acc', 0.8703703703703703)
pass train loss now ('acc', 0.8703703703703703)
vaild loss ('acc', 0.5869652139144342)
pass vaild loss ('acc', 0.6327160493827161)
pass vaild loss now ('acc', 0.6327160493827161)
test loss ('acc', 0.62123)
pass test loss ('acc', 0.5938485804416404)
pass test loss now ('acc', 0.5938485804416404)
not pass

2019-08-22 15:47:51 layer: 3
all data ('acc', 0.9063174730107957)
[p:272/1:130|n:24738/1:10469] 
train loss ('acc', 0.9063174730107957)
pass train loss ('acc', 0.8933823529411765)

KeyboardInterrupt: 